<a href="https://colab.research.google.com/github/matlogica/AADC-Python/blob/main/getting-started/06-stoch_interp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import sys
#!pip install https://matlogica.com/DemoReleases/aadc-1.7.5.26-cp3{sys.version_info.minor}-cp3{sys.version_info.minor}-linux_x86_64.whl
import aadc
import numpy as np
import random


In [14]:
random.seed(42)
from datetime import date, timedelta
from dateutil.relativedelta import relativedelta

# Get today's date
today_date = date.today()
dates = np.array([today_date + relativedelta(years=i) for i in range(21)])
times = aadc.array([(date - today_date).days / 365 for date in dates])

kernel = aadc.Kernel()
kernel.start_recording()

zero_rates = aadc.array([0.0025 + 0.005 * 0.02 * random.randint(0, 99) for i in range(len(dates))])
zero_args = zero_rates.mark_as_input()

dfs = np.exp(-zero_rates * times)
t = aadc.idouble(5.0)
arg_t = t.mark_as_input()
df = np.interp(t, times, dfs)

df_result = df.mark_as_output()
print(df)

kernel.stop_recording()
kernel.print_passive_extract_locations()


You are using evaluation version of AADC. Expire date is 20240901
idouble([AAD[rv] [adj] :83,9.72e-01])
Number active to passive conversions: 0 while recording Python


In [15]:
def get_aadc_risks(t):
    request = { df_result: zero_args }
    r = aadc.evaluate(kernel, request, { arg_t: [t] }, aadc.ThreadPool(1))
    return [item for subdict in r[1].values() for sublist in subdict.values() for item in sublist]

get_aadc_risks(5.0)

[0.0,
 0.0,
 0.0,
 0.0,
 -0.010966410208294638,
 -4.989033589791705,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

In [16]:
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interactive


def risk_plot(t):
    risks = get_aadc_risks(t)

    plt.figure(figsize=(10, 6))
    plt.bar(range(len(times)), risks)
    plt.xlabel('Years')
    plt.ylabel('Sensitivity')
    plt.title('Interpolated curve delta')
    plt.show()

    # Create interactive sliders for level and slope
t_slider = widgets.FloatSlider(
    value=5.0,
    min=0.0,
    max=20.0,
    step=0.2,
    description='Time: ',
    continuous_update=True
)

# Use interactive function to update the plot
interactive_plot = interactive(risk_plot, t=t_slider)
output = interactive_plot.children[-1]
output.layout.height = '600px'
interactive_plot


interactive(children=(FloatSlider(value=5.0, description='Time: ', max=20.0, step=0.2), Output(layout=Layout(h…